In [59]:
import json, collections, time
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="location_converter")

# Read the JSON file
with open('raw.json', 'r') as f:
    data = json.load(f)

In [67]:
with open('../locations.json', 'r') as f:
    location_cache = json.load(f)


def get_coord(location):
    if location_cache.get(location) is not None:
        return location_cache[location]

    print(f"API Call! for {location}")
    geolocator = Nominatim(user_agent="location_converter")
    time.sleep(1)

    loc = geolocator.geocode(location)
    assert loc, f"Location not found, {loc.raw}"
    
    location_cache[location] = [loc.latitude, loc.longitude, loc.address]
    return location_cache[location]

location_coordinates = {}

for i in data:
    location = i['job_posting_location']
    #####################
    if "remote" in location.lower():
        location_coordinates[location] = [["remote", "remote", "remote"]]
        continue
    #####################
    split_by = " and "
    if split_by in location:
        location_coordinates[location] = []
        for loc in location.split(split_by):
            if "more" in loc:
                continue
            coord = get_coord(loc)
            location_coordinates[location].append(coord)
        continue
    #####################
    split_by = " | "
    if split_by in location:
        location_coordinates[location] = []
        for loc in location.split(split_by):
            coord = get_coord(loc)
            location_coordinates[location].append(coord)
        continue
    #####################
    split_by = ";"
    if split_by in location:
        locs = []
        for loc in location.split(split_by):
            coord = get_coord(loc)
            locs.append(coord)
        location_coordinates[location] = locs
        continue
    split_by = " • "
    if split_by in location:
        locs = []
        for loc in location.split(split_by):
            coord = get_coord(loc)
            locs.append(coord)
        location_coordinates[location] = locs
        continue
    # #####################
    # if "locations" in location.lower():
    #     # print(location)
    #     continue
    removals = ["Location pin icon ", "locations", "office", "Category"]
    for i in removals:
        location = location.replace(i, "")
    try:
        location_coordinates[location] = [get_coord(location)]
    except Exception as e:
        print(f"Error: {location}")
        print(e)

with open('../locations.json', 'w') as f:
    location_coordinates.update(location_cache)
    json.dump(location_coordinates, f, indent=2)

API Call! for Austin, Texas Metropolitan Area
Error: Austin, Texas Metropolitan Area
'NoneType' object has no attribute 'raw'
API Call! for San Jose, California, United States of America Austin, Texas, United States of America San Francisco
Error: San Jose, California, United States of America Austin, Texas, United States of America San Francisco
HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=San+Jose%2C+California%2C+United+States+of+America+Austin%2C+Texas%2C+United+States+of+America+San+Francisco&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
API Call! for San Francisco Office
Error: San Francisco Office
'NoneType' object has no attribute 'raw'
API Call! for  Schnucks Store Support Center (Corporate Office)
Error:  Schnucks Store Support Center (Corporate Office)
'NoneType' object has no attribute 'raw'
API Call! for US

In [68]:

with open('../locations.json', 'w') as f:
    location_coordinates.update(location_cache)
    json.dump(location_coordinates, f, indent=2)